# Finding a Realtor using DATA
The objective is to use data from recent house sales to pick a realtor to list a house.
A good listing agent will get you a good price for your house in a reasonable amount of time. It matters less what kind of car they drive, what their lawn signs look like, or how they dress.

The most common way of finding a realtor is through referrals from friends and neighbors. Here instead we'll use *data* to narrow down the list of possibilities.

This notebook *scrapes* data from recently sold listings on zillow to grab the basic parameters of the houses along with information about the listing agent. While zillow does have a published API, it doesn't expose enough information to actually do this work. So we have to scrape. Internally zillow uses GraphQL but this modern API is not exposed for use by other applications.

![Dunphy Realtor](https://static1.srcdn.com/wordpress/wp-content/uploads/2021/02/modern-family-phil-dunphy-real-estate-bench.jpg)

The basic approach is:

1. Start with a search run by the user. All the parameters of the search will end up becoming URL parameters which makes it easy to then `GET` the search results page.
1. Grab the basic information for each listing on that page
1. Loop over the number of pages in the search results (could anywhere from 1 to hundreds, you might want to limit the number of listings you're after)
1. For each search result, perform additional GraphQL queries to get data that is not presented on the page by default. This mimics what happens when the user clicks on a zillow detail page section to expand it.
1. Make sure the scraping looks as much as possible like human behavior to avoid getting blocked

Some of the code here was adapted from [this gist](https://gist.github.com/scrapehero/5f51f344d68cf2c022eb2d23a2f1cf95).

In [1]:
# start by defining the zillow search url to be used as the starting point for scraping
searchUrl = "https://www.zillow.com/homes/recently_sold/house_type/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Emerald%20Lake%20Hills%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.28363290193751%2C%22east%22%3A-122.24844231966212%2C%22south%22%3A37.4409911163216%2C%22north%22%3A37.48306057333994%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A15%7D"

This search corresponds to this map, with each yellow dot representing a sale.
![map](./Screen%20Shot%202021-05-11%20at%2011.22.09%20AM.png)

In [2]:
# for testing with a smaller list of homes
testUrl = 'https://www.zillow.com/homes/recently_sold/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Emerald%20Lake%20Hills%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.26539028953387%2C%22east%22%3A-122.24835289787127%2C%22south%22%3A37.46707694546323%2C%22north%22%3A37.477856803154694%7D%2C%22mapZoom%22%3A16%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A2000000%2C%22max%22%3A3500000%7D%2C%22doz%22%3A%7B%22value%22%3A%226m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22min%22%3A7249%2C%22max%22%3A12517%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'

In [3]:
from lxml import html
import requests
import json
from datetime import datetime
from random import randint
from time import sleep
import pandas as pd

In [4]:
def clean(text):
    # Utility function to clean the 'json as comment' that comes back from zillow
    if text:
        return ' '.join(' '.join(text).split()).replace('<!--', "").replace("-->", "")
    return None

In [5]:
def save_json(data,fname):
    with open(fname, 'w') as outfile:
        json.dump(data, outfile)

In [6]:
def appendScalarsFromDict(a,b):
    """
    Copies all the scalar keys (strings, ints, floats, boolean, datetimes) from dictionary a to dictionary b
    """
    for key in a:  
        if isinstance(a[key],(str,int,float,bool,datetime)):
            b[key] = a[key]

In [7]:
def clean_results(response):
    """
    Extract the total number of pages from a Zillow search page
    along with the json object for the results
    
    Parameters:
        response: HTTP response object
    
    Returns:
        listResults json of results (an array, usually up to 40 results)
        totalPages the number of total pages returned by the search
        cookies the cookies from the response
    """
    
    parser = html.fromstring(response.text)
    jblob = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
    cleaned_data = clean(jblob)
    try:
        json_data = json.loads(cleaned_data)
        save_json(json_data,'cleaned_data.json')
        totalPages = json_data.get('cat1').get('searchList').get('totalPages')
        listResults = json_data.get('cat1').get('searchResults').get('listResults', [])
        return listResults, totalPages, response.cookies
    except ValueError:
        print("Invalid json")
        return None

In [8]:
def get_json_from_search(response):
    """
    Given a response from a Zillow search extract the total number of pages 
    along with the json object for the page
    
    Parameters:
    response HTTP response object
    
    Returns:
    listResults json of results (an array, usually up to 40 results)
    totalPages the number of total pages returned by the search
    """
    parser = html.fromstring(response.text)
    print("parsing json data")
    jblob = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
    cleaned_data = clean(jblob).replace('<!--', "").replace("-->", "")
    try:
        json_data = json.loads(cleaned_data)
        json.save(json_data)
        totalPages = json_data.get('searchList').get('totalPages')
        print('Total pages: ',totalPages)
        listResults = json_data.get('searchResults').get('listResults', [])
        return listResults, totalPages
    except ValueError:
        print("Invalid json")
        return None

In [9]:
def get_headers(referer=None):
    """
    Returns the headers required to execute a Zillow search request"
    
    These headers were copied from an actual web request to zillow using Chrome's inspector
    Without the right headers Zillow will perform a search around your geolocated IP address
    """
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,fr;q=0.8',
        'dnt': '1',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36' 
    }
    if referer:
        headers['referer'] = referer
    return headers

In [25]:
def get_response(uri, retries=5, cookies=None, referer=None):
    """
    Perform a GET request and return the response
    
    Parameters:
        uri uri to GET
        retries number of times to retry
    
    Returns: an HTTP response object
    """
    for i in range(retries): # try up to 5 times
        sleep(randint(5,10)) # act human
        response = requests.get(uri, headers=get_headers(referer), cookies=cookies)
        print(response.status_code,'GET ',uri[:uri.find('?')],'…')
        if not response.ok:
            save_to_file(response) # for debugging
            continue # retry
        else:
            return response
    return None

In [11]:
def get_gql_headers(zpid):
    """
    Return the GraphQL headers required to query Zillow's GraphQL server
    
    Parameters:
        zpid the Zillow property ID
    
    Returns: a JSON POST header object
    """
    # copied from a qraphQL request to zillow using Chrome
    return {
        'authority': 'www.zillow.com',
        'method': 'POST',
        'path': '/graphql/?zpid={}'.format(zpid),
        'scheme': 'https',
        'accept': '*/*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en',
        'content-type': 'text/plain',
        'dnt': '1',
        'origin': 'https://www.zillow.com',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }

In [12]:
def PriceTaxQuery(zpid, clientVersion="home-details/6.0.11.1315.master.2fc8ca5",timePeriod="FIVE_YEARS",metricType="LOCAL_HOME_VALUES",forecast=True):
    return {
        "query": """
             query PriceTaxQuery($zpid: ID!, $metricType: HomeValueChartMetricType, $timePeriod: HomeValueChartTimePeriod) {
                 property(zpid: $zpid) {
                    address {
                        city
                        community
                        neighborhood
                        state
                        streetAddress
                        subdivision
                        zipcode
                    }
                    bathrooms
                    bedrooms
                    countyFIPS
                    dateSold
                    hdpUrl
                    homeStatus
                    homeValueChartData(metricType: $metricType, timePeriod: $timePeriod) {
                        points {
                            x
                            y
                        }
                        name
                    }
                    lastSoldPrice
                    latitude
                    livingArea
                    livingAreaUnits
                    longitude
                    lotSize
                    parcelId
                    price
                    priceHistory {
                        time
                        price
                        event
                        buyerAgent {
                            profileUrl
                            name
                        }
                        sellerAgent {
                            profileUrl
                            name
                        }
                    }
                    yearBuilt
                    zestimate
                    zpid
                }
            }
        """,
        "operationName": "PriceTaxQuery",
        "variables": {
            "zpid": zpid,
            "timePeriod": timePeriod,
            "metricType": metricType,
            "forecast": forecast
        },
        "clientVersion": clientVersion
    }

In [13]:
def post_qql_query(zpid,query,gql_server = 'https://www.zillow.com/graphql/'):
    """
    Run the PriceTaxQuery query against Zillow's GraphQL server
    
    Parameters:
        zpid Zillow property ID
    
    Returns: json object including all the data requested by the query 
             while omitting the top two levels of hierarchy (data.property)
    """

    
    sleep(randint(1,4)) # act human
    r = requests.post(
        gql_server,
        json=query(zpid),
        headers=get_gql_headers(zpid),
        params={'zpid': zpid}
    )

    print(r.status_code,'POST ',gql_server,zpid)

    if r.ok:
        try:
            return r.json().get('data').get('property')
        except:
            print('Error getting json from graphql query results')
            return None
    else:
        printErrors(r)
        return None

In [14]:
def pretty_print_POST(req):
    """
    Pretty Print a POST request
    
    Parameters:
        req the POST request built using requests.Request and prepared with .prepare()
    
    Pay attention at the formatting used in this function because it is programmed
    to be pretty printed and may differ from the actual request.
    """
    print('{}\n{}\r\n{}\r\n\r\n{}'.format(
        '-----------START-----------',
        req.method + ' ' + req.url,
        '\r\n'.join('{}: {}'.format(k, v) for k, v in req.headers.items()),
        req.body,
    ))

In [15]:
def test_PriceTaxQuery(zpid,refererUrl,cookies,queryName='PriceTaxQuery'):
    """
    Used to debug the graphql POST requests
    See https://stackoverflow.com/a/23816211/2805154
    
    Parameters:
        zpid zillow property id
        refererUrl the url of the zillow property page
        cookies cookies set by the zillow server on a previous request
    """
    
    req = requests.Request('POST',
        'https://www.zillow.com/graphql/',
        json=compose_gql_query(zpid,queryName),
        headers=get_gql_headers(zpid),
        params=params,
        cookies=cookies
    )
    prepared = req.prepare()
    pretty_print_POST(prepared)

In [16]:
def get_zEstimate_at_listing(homeValueChartData,listedAt,dateSold):
    """
    Get the history of zEstimates for this property and pick out the one that
    is just prior to the listing date (if it exists) or the sell date
    
    Parameters:
        homeValueChartData: array of x,y points corresponding to time and zEstimatezpid zillow property id
        listedAt listing date (can be null)
        dateSold (should not be null)
    
    Returns an object with keys:
        zEstimate_at_listing the zEstimate at the earliest of (listing date,dateSold) or None
        zEstimate_date the date corresponding to the zEstimate_at_listing
    """

    first_listing_date = zEstimate_at_listing = zEstimate_date = None

    if isinstance(dateSold,datetime) & isinstance(listedAt,datetime):
        first_listing_date = min([dateSold,listedAt])
    elif isinstance(dateSold,datetime):  
        first_listing_date = dateSold
    elif isinstance(listedAt,datetime):
        first_listing_date = listedAt
    
    if first_listing_date:
        for el in homeValueChartData:
            if el.get('name') == "This home":
                points = el.get('points')
                # points are in ascending time order
                # loop until we've exceeded the first_listing_date
                # keep the last values before we go past the first_listing_date
                for p in points: 
                    x = datetime.fromtimestamp(p.get('x')/1000)
                    if x < first_listing_date:
                        zEstimate_date = x
                        zEstimate_at_listing = p.get('y')
                    else:
                        break
    return {
        "zEstimate_at_listing": zEstimate_at_listing, 
        "zEstimate_date": zEstimate_date
    }

In [17]:
def get_listing_info(priceHistory,dateSold):
    """
    Get the price history for the property and extract key pieces of data
    
    Parameters:
        priceHistory: array of objects containing pricing events
    
    Returns an object with keys:
        sellerAgent
        sellerAgentUrl
        listedAt
        firstListingPrice
    """
    sellerAgent = sellerAgentUrl = buyerAgent =  buyerAgentUrl = listedAt = firstListingPrice = None
    foundSoldEvent = False
    timesListed = 0

    # loop through the price history until a sold event is found (typically the first event)
    for ev in priceHistory:
        eventTime = datetime.fromtimestamp(ev.get('time')/1000)
        if (not foundSoldEvent) & (ev.get('event') == 'Sold'):
            foundSoldEvent = True

            # get the seller's agent info
            sa = ev.get('sellerAgent')
            if sa:
                sellerAgent = sa.get('name')
                sellerAgentUrl = 'https://www.zillow.com' + sa.get('profileUrl')

            # get the buyer's agent info
            ba = ev.get('buyerAgent')
            if ba:
                buyerAgent = ba.get('name')
                buyerAgentUrl = 'https://www.zillow.com' + ba.get('profileUrl')

        if foundSoldEvent & (ev.get('event') == 'Listed for sale'):
            # keep looking back at listing events for up to 9 months to try to find the earliest one matching the sale
            if (dateSold - eventTime).days < 9*30:
                timesListed += 1
                listedAt = eventTime
                firstListingPrice = ev.get('price')
    return { 
        "sellerAgent": sellerAgent, 
        "sellerAgentUrl": sellerAgentUrl, 
        "buyerAgent": buyerAgent, 
        "buyerAgentUrl": buyerAgentUrl, 
        "listedAt": listedAt, 
        "firstListingPrice": firstListingPrice, 
        "timesListed":timesListed
    }

In [18]:
def get_df_from_json(search_results):
    """
    Iterate over the JSON containing the search results to create a dataframe
    
    Parameters:
        search_results: json of search results.
        
        The only important part for us is the zpid 
        (Zillow property ID) of each property
    
    Returns: pandas dataframe of Zillow properties including specific fields
    """
    properties_list = []

    for i,properties in enumerate(search_results):
        data = {}

        zpid = properties.get('zpid')
        prop = post_qql_query(zpid,PriceTaxQuery)
        
        appendScalarsFromDict(prop,data)
        
        data['dateSold'] = datetime.fromtimestamp(data['dateSold']/1000)

        # promote address sub-properties
        appendScalarsFromDict(prop['address'],data)
        
        # extract recent sales history and flatten
        appendScalarsFromDict(get_listing_info(prop['priceHistory'],data['dateSold']),data)
                    
        # extract zEstimate right before the listing and flatten
        if 'listedAt' in data:
            appendScalarsFromDict(get_zEstimate_at_listing(prop['homeValueChartData'],data['listedAt'],data['dateSold']),data)
        
        properties_list.append(data)
    return pd.DataFrame(properties_list)

In [19]:
def insert_page_number(searchUrl,pageNumber):
    """
    
    Parameters:
        searchUrl the original search URL
        pageNumber an integer in [1,totalPages]
    
    Returns a paginated url based on the initial searchUrl which has a default page 1

    """
    p = 'pagination%22%3A%7B%22currentPage%22%3A{}%7D%2C%22'.format(pageNumber)
    return searchUrl.replace('pagination%22%3A%7B%7D%2C%22',p)

In [20]:
def scrape(url):
    """
    starting with a Zillow search URL, traverse each page and scrape the results
    
    Parameters:
        url search url composed through the Zillow web UI\
    
    Returns a pandas dataframe of results
    """
    response = get_response(url)

    if not response:
        print("Failed to fetch the page, please check `response.html` to see the response received from zillow.com.")
        return None

    results, totalPages, cookies = clean_results(response)
    print('Total pages: ',totalPages)
    
    df = get_df_from_json(results)
    prevUrl = url
    
    for p in range(2,totalPages+1):
        nextUrl = insert_page_number(url,p)
        response = get_response(nextUrl,cookies=cookies,referer=prevUrl)
        results, _, cookies = clean_results(response)
        df = df.append(get_df_from_json(results),ignore_index=True)
        prevUrl = nextUrl
    df.set_index('zpid',inplace=True)
    return df

In [26]:
%%time
df = scrape(testUrl)
df.tail()

200 GET  https://www.zillow.com/homes/recently_sold/ …
Total pages:  1
200 POST  https://www.zillow.com/graphql/ 15571474
200 POST  https://www.zillow.com/graphql/ 15573074
200 POST  https://www.zillow.com/graphql/ 15574389
200 POST  https://www.zillow.com/graphql/ 2076436147
200 POST  https://www.zillow.com/graphql/ 15574421
200 POST  https://www.zillow.com/graphql/ 15571456
200 POST  https://www.zillow.com/graphql/ 15574442
200 POST  https://www.zillow.com/graphql/ 89238984
200 POST  https://www.zillow.com/graphql/ 15571150
200 POST  https://www.zillow.com/graphql/ 15573051
200 POST  https://www.zillow.com/graphql/ 15571345
200 POST  https://www.zillow.com/graphql/ 15574491
200 POST  https://www.zillow.com/graphql/ 15571438
200 POST  https://www.zillow.com/graphql/ 15575072
CPU times: user 399 ms, sys: 126 ms, total: 525 ms
Wall time: 53.4 s


,bathrooms,bedrooms,countyFIPS,dateSold,hdpUrl,homeStatus,lastSoldPrice,latitude,livingArea,livingAreaUnits,...,zipcode,buyerAgent,buyerAgentUrl,listedAt,firstListingPrice,timesListed,zEstimate_at_listing,zEstimate_date,sellerAgent,sellerAgentUrl
9,4.0,4.0,06081,2021-02-17 16:00:00,/homedetails/194-Hillview-Ave-Redwood-City-CA-...,RECENTLY_SOLD,2900000,37.476879,2310.0,Square Feet,...,94062,Mary Sanbrook,https://www.zillow.com/profile/Mary-Sanbrook/,2021-01-07 16:00:00,2825000,1,2595356.0,2020-12-31,Stacey Jamison,https://www.zillow.com/profile/sjamison05/
10,2.0,3.0,06081,2021-01-24 16:00:00,/homedetails/579-Canyon-Rd-Redwood-City-CA-940...,RECENTLY_SOLD,2156000,37.472820,1860.0,Square Feet,...,94062,Lisa Karson,https://www.zillow.com/profile/lisa-karson/,2020-10-06 17:00:00,2300000,1,2153428.0,2020-09-30,Vicky Costantini,https://www.zillow.com/profile/vcostantini/
11,3.0,4.0,06081,2020-12-30 16:00:00,/homedetails/3091-Oak-Knoll-Dr-Redwood-City-CA...,RECENTLY_SOLD,2015000,37.474731,3154.0,Square Feet,...,94062,Hurchanik Team,https://www.zillow.com/profile/Hurchanik-Team/,2020-10-28 17:00:00,1995000,1,2411697.0,2020-09-30,Kristjan Higdon,https://www.zillow.com/profile/kristjanhigdon/
12,3.0,4.0,06081,2020-12-08 16:00:00,/homedetails/790-Bain-Pl-Redwood-City-CA-94062...,RECENTLY_SOLD,2425000,37.469337,2710.0,Square Feet,...,94062,Susan Kockos,https://www.zillow.com/profile/Susan-Kockos/,2020-10-16 17:00:00,2498000,1,2335891.0,2020-09-30,NaN,NaN
13,4.0,5.0,06081,2020-11-11 16:00:00,/homedetails/3407-Devon-Way-Redwood-City-CA-94...,RECENTLY_SOLD,3050000,37.468037,3105.0,Square Feet,...,94061,Carrie Davis,https://www.zillow.com/profile/carrie-davis/,2020-07-31 17:00:00,3198000,1,1304781.0,2020-07-31,Richard Kuhr,https://www.zillow.com/profile/KuhrProperties/


In [24]:
df.columns

Index(['bathrooms', 'bedrooms', 'countyFIPS', 'dateSold', 'hdpUrl',
       'homeStatus', 'lastSoldPrice', 'latitude', 'livingArea',
       'livingAreaUnits', 'longitude', 'lotSize', 'parcelId', 'price',
       'yearBuilt', 'zestimate', 'city', 'state', 'streetAddress', 'zipcode',
       'buyerAgent', 'buyerAgentUrl', 'listedAt', 'firstListingPrice',
       'timesListed', 'zEstimate_at_listing', 'zEstimate_date', 'sellerAgent',
       'sellerAgentUrl'],
      dtype='object')